# Goals and Steps

**Goal:**

Use Reddit's Trending section to discover if of the topics that are trending are discusssing stocks. Able to figure out if the trending section includes stocks by looking for a ticker symbol or any other keywords.

From this, look to see if the topic is discussing if the stock will go up or down, or if it's positive or negative talks about said stock.

**Steps:**

1. Obtain reddit API and do a GET to obtain the Trending section words. Also able to just scrape the words on the Trending section.

2. Use the Trending Title to figure out if the topic involves stocks or not.

3. After targeting Trending Titles that discuss stocks, use the title words or topic texts to see if these are positives or negatives about the stock (sentiment analysis).

Extras: (remove exact same comments posted multiple times, the same account spamming with one ticker etc.) You can also find the most mentioned stock and pick from an array of those options and see the comments/sentiments around that stock. Will also need an api that grabs all stock tickers, this will be used to loop through Reddit comments to find any mentions of tickers.

**What to grab when working with the [Reddit API](https://www.reddit.com/dev/api/):**"
In order to obtain trending subreddits, will have to go to the listings section of the api documentation, hyperlink, and look a the [/api/trending_subreddits](https://www.reddit.com/dev/api#GET_api_trending_subreddits).

Able to use the [live threads](https://www.reddit.com/dev/api#section_live) section to grab live threads.

**The subreddit to focus on is r/wallstreetbets**


**Reddit Sections:**
- "Best" is the highest upvote to downvote ratio

- "Top" are the most votes, upvotes and downvotes

- "Hot" is the most upvotes recently.


**Reddit Type Prefixes**
- t1_	- Comment

- t2_	- Account

- t3_	- Link

- t4_	- Message

- t5_	- Subreddit

- t6_	- Award

# Imports

In [1]:
import pandas as pd
import numpy as np
import os

# API, scrappping, etc.
from bs4 import BeautifulSoup
import requests

In [2]:
# Reddit API Credentials
CLIENT_ID = os.environ.get("REDDIT_CLIENT_ID")
SECRET_KEY = os.environ.get("REDDIT_SECRET_KEY")
REDDIT_PASSWORD = os.environ.get("REDDIT_PASSWORD")

In [3]:
# Requests a temporary auth token from Reddit
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

In [4]:
# Retrieving access token
data = {
    "grant_type": "password",
    "username": "Thisguycodes2",
    "password": REDDIT_PASSWORD
        }

In [5]:
headers = {'User-Agent': 'redditAPITrending:myredditapp:0.0.1 (by /u/Thisguycodes2)'}

In [6]:
# Send request for API token
res = requests.post("https://www.reddit.com/api/v1/access_token",
                   auth=auth, data=data, headers=headers)
ACCESS_TOKEN = res.json()["access_token"]  # Token will be put in header to access API
headers = {**headers, **{'Authorization': f"bearer {ACCESS_TOKEN}"}}  # Adding authorization to our headers dictionary

# Using Reddit API On Trending Stock /r/wallstreetbets

In [7]:
def get_trending_subreddit(subreddit="wallstreetbets", hot_new_or_rising="hot", limit=25):
    """
    Able to select which subreddit
    you'd like to view, and from there
    you're able to view all the data/text
    in said subreddit.
    
    The max limit is 100. The limit will
    return data points.
    
    Able to run/call this, and it will 
    update it with the latest information
    """
    trending_posts = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{hot_new_or_rising}", headers=headers, params={"limit": limit})
    return trending_posts.json()["data"]["children"]

get_trending_subreddit()

[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'wallstreetbets',
   'selftext': 'Your daily trading discussion thread. Please keep the shitposting to a minimum. \n\n^Navigate ^WSB|^We ^recommend ^best ^daily ^DD\n:--|:--                                 \n**DD** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADD) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=week)\n**Discussion** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADiscussion) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADiscussion&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADiscussion&amp;restrict_sr=on&amp;t=week)

In [8]:
subreddit = "wallstreetbets"
trending_posts = requests.get(f"https://oauth.reddit.com/r/{subreddit}/hot", headers=headers, params={"limit": 100})
stocks_df = pd.DataFrame()
for post in trending_posts.json()["data"]["children"]:
    stocks_df = stocks_df.append({
        "Subreddit": post["data"]["subreddit"],
        "Title": post["data"]["title"],
        "Text": post["data"]["selftext"],
        "Up Vote Ratio": post["data"]["upvote_ratio"],
        "Up Votes": post["data"]["ups"],
        "Down Votes": post["data"]["downs"],
        "Thread_id": post["kind"] + "_" + post["data"]["id"]  # Also available by doing post["name"]
                }, ignore_index=True, sort=False)

In [9]:
stocks_df

,Down Votes,Subreddit,Text,Thread_id,Title,Up Vote Ratio,Up Votes
0,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0
1,0.0,wallstreetbets,,t3_luzpgy,It seems like you already know where,0.98,6875.0
2,0.0,wallstreetbets,,t3_lusqh7,If GME was a sitcom,0.95,24020.0
3,0.0,wallstreetbets,,t3_luv2to,Is this how one YOLOs? $200K on 400 x $800 Mar...,0.97,6969.0
4,0.0,wallstreetbets,,t3_luu7hm,This will age well.,0.98,4606.0
...,...,...,...,...,...,...,...
96,0.0,wallstreetbets,,t3_ltuu37,$3M loss....got a call yesterday to say that a...,0.97,5653.0
97,0.0,wallstreetbets,[Link to pt I](https://www.reddit.com/r/wallst...,t3_luuige,NVAX part II - Last call before a shit ton of ...,0.75,23.0
98,0.0,wallstreetbets,,t3_luinov,"🦍🦍🦍 Strong, 🦍🦍🦍 Never Sell. My two accounts wi...",0.85,128.0
99,0.0,wallstreetbets,I really resisted my urge to not do so but fin...,t3_lua3b7,Bought another 500 shares this week from IRA. ...,0.94,469.0


## Getting Reddit Thread Comments

In [10]:
# Looking at subreddit comments
subreddit = "wallstreetbets"
article = stocks_df["Thread_id"][0][3:]  # Returning only the id, not the kind (e.g. t3)
trending_posts_comments = requests.get(f"https://oauth.reddit.com/r/{subreddit}/comments/{article}", 
                                       headers=headers, 
                                       params={"limit":100, "sort": "top"})

In [11]:
trending_posts_comments.json()

[{'kind': 'Listing',
  'data': {'modhash': None,
   'dist': 1,
   'children': [{'kind': 't3',
     'data': {'approved_at_utc': None,
      'subreddit': 'wallstreetbets',
      'selftext': 'Your daily trading discussion thread. Please keep the shitposting to a minimum. \n\n^Navigate ^WSB|^We ^recommend ^best ^daily ^DD\n:--|:--                                 \n**DD** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADD) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADD&amp;restrict_sr=on&amp;t=week)\n**Discussion** | [All](https://reddit.com/r/wallstreetbets/search?sort=new&amp;restrict_sr=on&amp;q=flair%3ADiscussion) / [**Best Daily**](https://www.reddit.com/r/wallstreetbets/search?sort=top&amp;q=flair%3ADiscussion&amp;restrict_sr=on&amp;t=day) / [Best Weekly](https://www.reddit.com/r

In [12]:
trending_posts_comments.json()[1]["data"]["children"][0]["data"]["body"]

"If you're submitting good DD and you think it's getting bounced too often, PM me with links to 1-2 good DD posts of yours and I can add you to the approved submitter list. It'll rat you out and ping me instead of removing your post.\n\nDo me a favor and follow our Twitter [@Official_WSB](https://twitter.com/Official_WSB). There are a couple larger accounts claiming to be us and it's causing issues, people are scamming. I'm gonna make a submission about it during the week to get people on board, then we can slide in DMs to get shit done for us.\n\nDid a ton of work on the bots this weekend, hoping we can handle big traffic surges even better now."

In [13]:
for comments in trending_posts_comments.json()[1]["data"]["children"]:
    print(comments["data"].get("body"))  # Retrieving the "body" key did not work, accessing the "body" key by using the .get method

If you're submitting good DD and you think it's getting bounced too often, PM me with links to 1-2 good DD posts of yours and I can add you to the approved submitter list. It'll rat you out and ping me instead of removing your post.

Do me a favor and follow our Twitter [@Official_WSB](https://twitter.com/Official_WSB). There are a couple larger accounts claiming to be us and it's causing issues, people are scamming. I'm gonna make a submission about it during the week to get people on board, then we can slide in DMs to get shit done for us.

Did a ton of work on the bots this weekend, hoping we can handle big traffic surges even better now.
It’s that time of day.. hype and upvote gme to confirm my bias 🚀🚀🙏
Gonna spread my risk: 50% buy more GME, 50% hold my GME.
Probably gonna panic hold
If I ever became super rich overnight, I wouldn't tell anybody or spend a cent. Then I'd start telling my friends and family that a Nigerian prince emailed me and I sent him $2000. Of course they'd th

In [14]:
# Looking at subreddit comments
comments_df = pd.DataFrame()
subreddit = "wallstreetbets"
for article_id in stocks_df["Thread_id"]:
    trending_posts_comments = requests.get(f"https://oauth.reddit.com/r/{subreddit}/comments/{article_id[3:]}", 
                                       headers=headers, 
                                       params={"limit":100, "depth":5})
    for comments in trending_posts_comments.json()[1]["data"]["children"]:
        comments_df = comments_df.append({
                                        "Thread_id": comments["data"].get("parent_id"),
                                        "Comments": comments["data"].get("body")
                                        }, ignore_index=True)


In [15]:
comments_df

,Comments,Thread_id
0,If you're submitting good DD and you think it'...,t3_luqc3p
1,NIO moon and then APHA Moon thanks,t3_luqc3p
2,Dudes out here with 5 shares spamming every th...,t3_luqc3p
3,If you or a loved one has been diagnosed with ...,t3_luqc3p
4,GME and AMC!!! 📈🚀🌑💎🙌🤑🍦🍗,t3_luqc3p
...,...,...
3368,I bought MRNA at 66 a share best thing ever. \...,t3_lulvbm
3369,ive been strapped into moderna for a while now...,t3_lulvbm
3370,What was your entry point? Link?,t3_lulvbm
3371,I generally agree with your DD. I wonder where...,t3_lulvbm


### Problem:

For some weird reason the key value is giving an error, saying that it is not there when infact it is.

### Solution:

Using the .get method, which is introduced when using dictionaries, was able to fix the problem.

In [20]:
# Merging both dataframes using the Thread_id column
test = pd.concat([stocks_df, comments_df])

In [21]:
test

,Down Votes,Subreddit,Text,Thread_id,Title,Up Vote Ratio,Up Votes,Comments
0,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0,NaN
1,0.0,wallstreetbets,,t3_luzpgy,It seems like you already know where,0.98,6875.0,NaN
2,0.0,wallstreetbets,,t3_lusqh7,If GME was a sitcom,0.95,24020.0,NaN
3,0.0,wallstreetbets,,t3_luv2to,Is this how one YOLOs? $200K on 400 x $800 Mar...,0.97,6969.0,NaN
4,0.0,wallstreetbets,,t3_luu7hm,This will age well.,0.98,4606.0,NaN
...,...,...,...,...,...,...,...,...
3368,NaN,NaN,NaN,t3_lulvbm,NaN,NaN,NaN,I bought MRNA at 66 a share best thing ever. \...
3369,NaN,NaN,NaN,t3_lulvbm,NaN,NaN,NaN,ive been strapped into moderna for a while now...
3370,NaN,NaN,NaN,t3_lulvbm,NaN,NaN,NaN,What was your entry point? Link?
3371,NaN,NaN,NaN,t3_lulvbm,NaN,NaN,NaN,I generally agree with your DD. I wonder where...


In [23]:
test2 = pd.merge(left=stocks_df, right=comments_df, how="inner", on="Thread_id")

In [24]:
test2

,Down Votes,Subreddit,Text,Thread_id,Title,Up Vote Ratio,Up Votes,Comments
0,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0,If you're submitting good DD and you think it'...
1,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0,NIO moon and then APHA Moon thanks
2,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0,Dudes out here with 5 shares spamming every th...
3,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0,If you or a loved one has been diagnosed with ...
4,0.0,wallstreetbets,Your daily trading discussion thread. Please k...,t3_luqc3p,"What Are Your Moves Tomorrow, March 01, 2021",0.95,1526.0,GME and AMC!!! 📈🚀🌑💎🙌🤑🍦🍗
...,...,...,...,...,...,...,...,...
3368,0.0,wallstreetbets,\*This will be my first DD post so let me know...,t3_lulvbm,Moderna (MRNA) DD – Why a 60B Company is Still...,0.71,66.0,I bought MRNA at 66 a share best thing ever. \...
3369,0.0,wallstreetbets,\*This will be my first DD post so let me know...,t3_lulvbm,Moderna (MRNA) DD – Why a 60B Company is Still...,0.71,66.0,ive been strapped into moderna for a while now...
3370,0.0,wallstreetbets,\*This will be my first DD post so let me know...,t3_lulvbm,Moderna (MRNA) DD – Why a 60B Company is Still...,0.71,66.0,What was your entry point? Link?
3371,0.0,wallstreetbets,\*This will be my first DD post so let me know...,t3_lulvbm,Moderna (MRNA) DD – Why a 60B Company is Still...,0.71,66.0,I generally agree with your DD. I wonder where...
